# Transcribe youtube audio to text

In [ ]:
# !pip install openai
# !pip install pytube -q

### Get audio from youtube video

In [ ]:
from pytube import YouTube
video_id = "hixrHmqf2zc"
youtube_video_url = f"https://www.youtube.com/watch?v={video_id}"
youtube_video = YouTube(youtube_video_url)

In [ ]:
title = youtube_video.title
title

In [ ]:
for stream in youtube_video.streams.filter(only_audio=True):
  print(stream)

In [ ]:
first_stream = youtube_video.streams.filter(only_audio=True).first()
first_stream

In [ ]:
import os
audio_dir = "audio"
if not os.path.exists(audio_dir):
    os.makedirs(audio_dir)

stream.download(filename=f'{audio_dir}/youtube_audio.mp3')

### Chunk into smaller parts


In [ ]:
# Use pydub to segment the audio file
!pip install pydub

In [ ]:
from pydub import AudioSegment

audio = AudioSegment.from_file("youtube_audio.mp3")

In [ ]:
audio_length = audio.duration_seconds
audio_length

In [ ]:
mins = 10 * 60 * 1000
file_counter = 1
start_time = 0

while start_time < audio_length * 1000:
    part = audio[start_time : start_time + mins]
    part.export(f"{audio_dir}/part{file_counter}.mp3", format="mp3")
    file_counter += 1
    start_time += mins

### Transcribe using Whisper API

In [ ]:
import openai
api_key = input("Enter your open ai api key: ")
openai.api_key = api_key

In [ ]:
transcript = ""
for count in range(1, file_counter):
  file = f"{audio_dir}/part{count}.mp3"
  audio_file= open(file, "rb")
  obj = openai.Audio.transcribe("whisper-1", audio_file)
  transcript += obj["text"]

Add a new paragrah every 3 sentences for easier reading.

Thoughts: How can we split into paragraphs naturally?

In [ ]:
structured_script = ""
num_sentence = 0
num_characters = 0

for c in transcript:
  structured_script += c
  if c == ".":
    num_sentence +=1
    if num_sentence >= 4 and num_characters >= 100:
      structured_script += "\n\n"
      num_sentence = 0
      num_characters = 0
  num_characters += 1

with open(f"{audio_dir}/structured_transcript.txt", 'w') as file:
  file.write(structured_script)

### Another option: use Youtube Transcribe API.

This works if the youtube video has subtitle enabled.


In [ ]:
# !pip install youtube-transcript-api

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi
YouTubeTranscriptApi.get_transcript(video_id)